#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [99]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [178]:
products = graphlab.SFrame('amazon_baby.gl/')

#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [179]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [180]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [181]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


In [182]:
graphlab.canvas.set_target('ipynb')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [183]:
#ignore all 3* reviews
products_new = products[products['rating'] != 3]

In [184]:
#positive sentiment = 4* or 5* reviews
products_new['sentiment'] = products_new['rating'] >=4

In [185]:
products_new.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1


##Let's train the sentiment classifier

In [186]:
train_data,test_data = products_new.random_split(.8, seed=0)

In [187]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.276348     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 4.476378     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 5.379562     | 0.92

#Evaluate the sentiment model

In [188]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.220637898687 | 0.00513661981879 | 28034 | 5330 |
 | 0.0010000000475  | 0.779362101313 |  0.994863380181  | 28034 | 5330 |
 | 0.00200000009499 | 0.740337711069 |  0.993543554256  | 28034 | 5330 |
 | 0.00300000002608 | 0.718198874296 |  0.992687450952  | 28034 | 5330 |
 | 0.00400000018999 | 0.703001876173 |  0.992116715417  | 28034 | 5330 |
 | 0.00499999988824 | 0.692120075047 |  0.991617321824  | 28034 | 5330 |
 | 0.00600000005215 | 0.682551594747 |  0.991046586288  | 28034 | 5330 |
 | 0.00700000021607 | 0.672420262664 |  0.990582863666  | 28034 | 5330 |
 | 0.00800000037998 | 0.662101313321 |  0.990333166869  | 28034 | 5330 |
 | 0.00899999961257 

In [189]:
sentiment_model.show(view='Evaluation')

#Week3 Assignment

In [191]:
#write a function called awesome_count which takes in the word counts 
#and returns the number of times ‘awesome’ appears in the reviews.
def awesome_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'awesome' in word_count_entry:
       return word_count_entry['awesome']
    else:
       return 0

In [192]:
#use .apply() to iterate awesome_count for each row of products[‘word_count’] 
#and create a new column called ‘awesome’ with the resulting counts.
products_new['awesome'] = products_new['word_count'].apply(awesome_count)

In [193]:
#function great_count
def great_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'great' in word_count_entry:
       return word_count_entry['great']
    else:
       return 0

In [194]:
products_new['great'] = products_new['word_count'].apply(great_count)

In [195]:
#function fantastic_count
def fantastic_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'fantastic' in word_count_entry:
       return word_count_entry['fantastic']
    else:
       return 0

In [196]:
products_new['fantastic'] = products_new['word_count'].apply(fantastic_count)

In [197]:
#function amazing_count
def amazing_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'amazing' in word_count_entry:
       return word_count_entry['amazing']
    else:
       return 0

In [198]:
products_new['amazing'] = products_new['word_count'].apply(amazing_count)

In [199]:
#function love_count
def love_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'love' in word_count_entry:
       return word_count_entry['love']
    else:
       return 0

In [200]:
products_new['love'] =products_new['word_count'].apply(love_count)

In [201]:
#function horrible_count
def horrible_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'horrible' in word_count_entry:
       return word_count_entry['horrible']
    else:
       return 0

In [202]:
products_new['horrible'] = products_new['word_count'].apply(horrible_count)

In [203]:
#function bad_count
def bad_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'bad' in word_count_entry:
       return word_count_entry['bad']
    else:
       return 0

In [204]:
products_new['bad'] = products_new['word_count'].apply(bad_count)

In [205]:
#function terrible_count
def terrible_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'terrible' in word_count_entry:
       return word_count_entry['terrible']
    else:
       return 0

In [206]:
products_new['terrible'] =products_new['word_count'].apply(terrible_count)

In [207]:
#function awful_count
def awful_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'awful' in word_count_entry:
       return word_count_entry['awful']
    else:
       return 0

In [208]:
products_new['awful'] = products_new['word_count'].apply(awful_count)

In [209]:
#function wow_count
def wow_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'wow' in word_count_entry:
       return word_count_entry['wow']
    else:
       return 0

In [210]:
products_new['wow'] = products_new['word_count'].apply(wow_count)

In [211]:
#function hate_count
def hate_count(word_count_entry):# word_count_entry is each row of products[‘word_count’]
    if 'hate' in word_count_entry:
       return word_count_entry['hate']
    else:
       return 0

In [212]:
products_new['hate'] =products_new['word_count'].apply(hate_count)

In [213]:
products_new.head()

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1,0


In [232]:
#Using the .sum() method on each of the new columns you created:
#selected_words = ['awesome', 'great', 'fantastic', 'amazing', 
#                  'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
print products_new['awesome'].sum()
print products_new['great'].sum()
print products_new['fantastic'].sum()
print products_new['amazing'].sum()
print products_new['love'].sum()
print products_new['horrible'].sum()
print products_new['bad'].sum()
print products_new['terrible'].sum()
print products_new['awful'].sum()
print products_new['wow'].sum()
print products_new['hate'].sum()

1057

###1.Which one is most used in the dataset? 
great
###2.Which one is least used? 
wow

#Create a new sentiment analysis model using only the selected_words as features

In [215]:
#Use the same train/test split as in the IPython Notebook from lecture:
train_data,test_data = products_new.random_split(.8, seed=0)

In [253]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 
                  'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [254]:
#Train a logistic regression classifier using just the selected_words
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                           target='sentiment',
                                                           features=selected_words,
                                                           validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.255343     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.419221     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.579700     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [255]:
#Examine the weights the learned classifier assigned to each of the 11 words in selected_words
word_weight = selected_words_model['coefficients']

In [261]:
word_weight = word_weight.sort('value',ascending=False)
print word_weight[0]
print word_weight[-1]

{'index': None, 'name': 'love', 'value': 1.399898343017463, 'class': 1}
{'index': None, 'name': 'terrible', 'value': -2.090499984872607, 'class': 1}


###3.Which one got the most positive weight?
love
###4.Which one got the most negative weight? 
terrible

#Comparing the accuracy of different sentiment analysis model

In [262]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+----------------+-------+------+
 |    threshold     |        fpr        |      tpr       |   p   |  n   |
 +------------------+-------------------+----------------+-------+------+
 |       0.0        | 0.000187336080929 |      0.0       | 27985 | 5338 |
 | 0.0010000000475  |   0.999812663919  |      1.0       | 27985 | 5338 |
 | 0.00200000009499 |   0.999625327838  |      1.0       | 27985 | 5338 |
 | 0.00300000002608 |   0.999625327838  |      1.0       | 27985 | 5338 |
 | 0.00400000018999 |   0.999437991757  | 0.999964266571 | 27985 | 5338 |
 | 0.00499999988824 |   0.999437991757  | 0.999964266571 | 27985 | 5338 |
 | 0.00600000005215 |   0.999437991757  | 0.999928533143 | 27985 | 5338 |
 | 0.00700000021607 |   0.999437991757  | 0.999928533143 | 27985 | 5338 |
 | 0.00800000037998 |   0.999437991757  | 0.999928533143 | 27985 | 5338 |
 | 0.008

In [263]:
selected_words_model.show(view='Evaluation')

In [248]:
#What is the accuracy majority class classifier on this task?
#Predicting every review is positive review,calculate positive review#/all reviews#
test_data['sentiment'].show(view='Categorical')

###5.The accuracy of the selected_words_model on the test_data：
0.84     # range from 0.841 to 0.871
###6.The accuracy of the sentiment_model：
0.909     # range from 0.901 to 0.931
###7.What is the accuracy majority class classifier on this task?
* majority class classifier simply predicts that every data point is from the most common class.
* 84.002%    # range from 0.811 to 0.841
###8.How do you compare the different learned models with the baseline approach where we are just predicting the majority class?
The model learned using all words performed much better than the other two. The other two approaches performed about the same.

#Investigate a product named ‘Baby Trend Diaper Champ’
#Interpreting the difference in performance between the models

In [264]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [265]:
#use the sentiment_model to predict the sentiment of each review in diaper_champ_reviews 
#and sort the results according to their ‘predicted_sentiment’.
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)
diaper_champ_reviews.head()

name,review,rating,word_count,predicted_sentiment
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1, 'less': 1,""friend's"": 1, '(which': ...",0.999999937267
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1, 'over': 1,'rweek': 1, 'sooo': 1, ...",0.999999917406
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2, 'just': 4,""don't"": 2, 'one,': 1, ...",0.999999899509
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1, 'able': 2,'over': 1, 'soon': 1, ...",0.999999836182
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",0.999999824745
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'son': 2, 'all': 1,'bags.': 1, 'son,': 1, ...",0.999999759315
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4, 'this': 3,'stink': 1, 'garbage' ...",0.999999692111
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1, 'all': 2,'bags.': 1, 'feedback': ...",0.999999642488
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1, '-': 3,'both': 1, 'results': 1, ...",0.999999604504
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'all': 1, 'humid': 1,'just': 1, 'less': 1, ...",0.999999486804


###9.What is the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model?
0.999999937267    # 0.9 to 1.0

In [241]:
diaper_champ_reviews[0:1]

name,review,rating,word_count,predicted_sentiment
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1, 'less': 1,""friend's"": 1, '(which': ...",0.999999937267


###10. In what range is the predicted_sentiment for this review, if we use the selected_words_model to analyze it?
0.7 to 0.8

In [274]:
# The result is wrong
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.8706971470401035]

In [249]:
diaper_champ_reviews['review'][0]

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

###11.Why is the value of the predicted_sentiment for the most positive review found using the sentiment_model much more positive than the value predicted using the selected_words_model?
None of the selected_words appeared in the text of this review.